In [1]:
import pandas as pd
import re
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
df=pd.read_csv('/content/drive/MyDrive/final_prj data/twitter_new.csv',encoding='latin-1')

In [3]:
df.shape

(1599999, 6)

In [4]:
df.rename(columns={"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D":'Tweet','_TheSpecialOne_':'Author','0':'Polarity'},inplace=True)

In [5]:
df

,Polarity,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,Author,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Polarity                      1599999 non-null  int64 
 1   1467810369                    1599999 non-null  int64 
 2   Mon Apr 06 22:19:45 PDT 2009  1599999 non-null  object
 3   NO_QUERY                      1599999 non-null  object
 4   Author                        1599999 non-null  object
 5   Tweet                         1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [7]:
pos_data=df[df['Polarity']==4]
neg_data=df[df['Polarity']==0]

In [8]:
pos_data=pos_data.iloc[:20000]
neg_data=neg_data.iloc[:20000]

In [9]:
dff=pd.concat([neg_data,pos_data],axis=0)
dff.reset_index(drop=True,inplace=True)

In [10]:
dff

,Polarity,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,Author,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
39995,4,1553584829,Sat Apr 18 14:39:51 PDT 2009,NO_QUERY,TempeHanna,We just met some awesome people at T.G.I Fridays
39996,4,1553584881,Sat Apr 18 14:39:52 PDT 2009,NO_QUERY,xSugarHIGH,oh my goodness . my feet are the most sensitiv...
39997,4,1553584934,Sat Apr 18 14:39:52 PDT 2009,NO_QUERY,vomitcupcakes,@hey_angy hahaha
39998,4,1553584954,Sat Apr 18 14:39:54 PDT 2009,NO_QUERY,StarrahPenzHitz,@Lamartian30 Pfft school is koo. I'm ranked 2n...


In [11]:
dff=dff[['Tweet','Polarity']]

In [12]:
dff['Tweet']=dff['Tweet'].str.lower()

<ipython-input-12-5f3daf934e26>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].str.lower()


In [13]:
dff['Polarity'].unique()

array([0, 4])

In [14]:
dff

,Tweet,Polarity
0,is upset that he can't update his facebook by ...,0
1,@kenichan i dived many times for the ball. man...,0
2,my whole body feels itchy and like its on fire,0
3,"@nationwideclass no, it's not behaving at all....",0
4,@kwesidei not the whole crew,0
...,...,...
39995,we just met some awesome people at t.g.i fridays,4
39996,oh my goodness . my feet are the most sensitiv...,4
39997,@hey_angy hahaha,4
39998,@lamartian30 pfft school is koo. i'm ranked 2n...,4


In [15]:
stopwords=set(stopwords.words('english'))

In [16]:
#Removing stopwords
def clean(text):
  words=text.split()
  words=[word for word in words if word.lower() not in stopwords]
  return ' '.join(words)

In [17]:
dff['Tweet']=dff['Tweet'].apply(clean)

<ipython-input-17-02f67842d65e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].apply(clean)


In [18]:
dff

,Tweet,Polarity
0,upset can't update facebook texting it... migh...,0
1,@kenichan dived many times ball. managed save ...,0
2,whole body feels itchy like fire,0
3,"@nationwideclass no, behaving all. i'm mad. he...",0
4,@kwesidei whole crew,0
...,...,...
39995,met awesome people t.g.i fridays,4
39996,oh goodness . feet sensitive part body . shit ...,4
39997,@hey_angy hahaha,4
39998,@lamartian30 pfft school koo. i'm ranked 2nd e...,4


In [19]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations

In [20]:
punctuations_list

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
#Removing punctuation
def clean_punc(text):
  return ''.join(x for x in text if x not in punctuations_list)

In [22]:
dff['Tweet']=dff['Tweet'].apply(lambda x:clean_punc(x))

<ipython-input-22-f9c200a828b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].apply(lambda x:clean_punc(x))


In [23]:
def extra_clean(text):
  c1=re.sub(r'(.)1+', r'1', text)                         #removing repeating characters
  c2=re.sub('((www.[^s]+)|(https?://[^s]+))',' ',c1)      #removing URLs
  c3=re.sub('[0-9]+', '',c2)                              #removing numeric numbers
  return c3

In [24]:
dff['Tweet']=dff['Tweet'].apply(lambda x:extra_clean(x))

<ipython-input-24-3816d22a5954>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].apply(lambda x:extra_clean(x))


In [25]:
dff

,Tweet,Polarity
0,upset cant update facebook texting it might cr...,0
1,kenichan dived many times ball managed save r...,0
2,whole body feels itchy like fire,0
3,nationwideclass no behaving all im mad here ca...,0
4,kwesidei whole crew,0
...,...,...
39995,met awesome people tgi fridays,4
39996,oh goodness feet sensitive part body shit fe...,4
39997,heyangy hahaha,4
39998,lamartian pfft school koo im ranked nd experim...,4


In [26]:
tokenizer=RegexpTokenizer(r'\w+')

In [27]:
dff['Tweet']=dff['Tweet'].apply(tokenizer.tokenize)

<ipython-input-27-773712fb40ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].apply(tokenizer.tokenize)


In [28]:
st=PorterStemmer()

In [29]:
def stemming(text):
  stemmed_words=[st.stem(word) for word in text]
  return ' '.join(stemmed_words)

In [30]:
dff['Tweet']=dff['Tweet'].apply(stemming)

<ipython-input-30-903d9c01da3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].apply(stemming)


In [31]:
lm=WordNetLemmatizer()

In [32]:
def lemma(text):
  words=text.split()
  lemma_words=[lm.lemmatize(word) for word in words]
  return ' '.join(lemma_words)

In [33]:
dff['Tweet']=dff['Tweet'].apply(lemma)

<ipython-input-33-95ca25bce22b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Tweet']=dff['Tweet'].apply(lemma)


In [34]:
dff

,Tweet,Polarity
0,upset cant updat facebook text it might cri re...,0
1,kenichan dive mani time ball manag save rest g...,0
2,whole bodi feel itchi like fire,0
3,nationwideclass no behav all im mad here cant ...,0
4,kwesidei whole crew,0
...,...,...
39995,met awesom peopl tgi friday,4
39996,oh good foot sensit part bodi shit feel orgasm...,4
39997,heyangi hahaha,4
39998,lamartian pfft school koo im rank nd experimen...,4


In [35]:
X=dff['Tweet']
y=dff['Polarity']
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [36]:
vec=TfidfVectorizer(ngram_range=(1,2), max_features=500000)

In [37]:
vec.fit(x_train)

TfidfVectorizer(max_features=500000, ngram_range=(1, 2))

In [38]:
x_train=vec.transform(x_train)
x_test=vec.transform(x_test)

In [39]:
models=[LogisticRegression(),DecisionTreeClassifier(),RandomForestClassifier(),SVC()]
for m in models:
  print('**********')
  print(m)
  model=m
  model.fit(x_train,y_train)
  y_pred=model.predict(x_test)
  acc_score=accuracy_score(y_test,y_pred)
  print(acc_score)
  print('**********')

**********
LogisticRegression()
0.744
**********
**********
DecisionTreeClassifier()
0.667125
**********
**********
RandomForestClassifier()
0.725875
**********
**********
SVC()
0.746625
**********
